## Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

### Principal Components Analysis

In [ ]:
o_dir = os.path.join('outputs','pca')
if os.path.isdir(o_dir) is not True:
    print("Creating '{0}' directory.".format(o_dir))
    os.mkdir(o_dir)

In [ ]:
pca = PCA()                               # Use all Principal Components
pca.fit(scdf)                             # Train model on all data
pcdf = pd.DataFrame(pca.transform(scdf))  # Transform data using model

for i in range(0,21):
    print("Amount of explained variance for component {0} is: {1:6.2f}%".format(i, pca.explained_variance_ratio_[i]*100))

print("The amount of explained variance of the SES score using each component is...")
sns.lineplot(x=list(range(1,len(pca.explained_variance_ratio_)+1)), y=pca.explained_variance_ratio_)

In [ ]:
pca = PCA(n_components=11)
pca.fit(scdf)
scores = pd.DataFrame(pca.transform(scdf), index=scdf.index)
scores.to_csv(os.path.join(o_dir,'Scores.csv.gz'), compression='gzip', index=True)

# Adapted from https://stackoverflow.com/questions/22984335/recovering-features-names-of-explained-variance-ratio-in-pca-with-sklearn
i = np.identity(scdf.shape[1])  # identity matrix

coef = pca.transform(i)

loadings = pd.DataFrame(coef, index=scdf.columns)
loadings.to_csv(os.path.join(o_dir,'Loadings.csv.gz'), compression='gzip', index=True)

In [ ]:
print(scores.shape)
scores.sample(5, random_state=42)

In [ ]:
print(loadings.shape)
loadings.sample(5, random_state=42)

In [ ]:
odf = pd.DataFrame(columns=['Variable','Component Loading','Score'])
for i in range(0,len(loadings.index)):
    row = loadings.iloc[i,:]
    for c in list(loadings.columns.values):
        d = {'Variable':loadings.index[i], 'Component Loading':c, 'Score':row[c]}
        odf = odf.append(d, ignore_index=True)

g = sns.FacetGrid(odf, col="Variable", col_wrap=4, height=3, aspect=2.0, margin_titles=True, sharey=True)
g = g.map(plt.plot, "Component Loading", "Score", marker=".")

### What Have We Done?

In [ ]:
sns.set_style('white')
sns.jointplot(data=scores, x=0, y=1, kind='hex', height=8, ratio=8)

#### Create an Output Directory and Load the Data

In [ ]:
o_dir = os.path.join('outputs','clusters-pca')
if os.path.isdir(o_dir) is not True:
    print("Creating '{0}' directory.".format(o_dir))
    os.mkdir(o_dir)

In [ ]:
score_df = pd.read_csv(os.path.join('outputs','pca','Scores.csv.gz'))
score_df.rename(columns={'Unnamed: 0':'lsoacd'}, inplace=True)
score_df.set_index('lsoacd', inplace=True)

# Ensures that df is initialised but original scores remain accessible
df = score_df.copy(deep=True)

score_df.describe()

In [ ]:
score_df.sample(3, random_state=42)

#### Rescale the Loaded Data

We need this so that differences in the component scores don't cause the clustering algorithms to focus only on the 1st component.

In [ ]:
scaler = preprocessing.MinMaxScaler()

df[df.columns] = scaler.fit_transform(df[df.columns])

df.describe()

In [ ]:
df.sample(3, random_state=42)